In [27]:
# imports
# packages used
import librosa
import os 

import math
import numpy as np
import pandas as pd
import ast
import re
# from sklearn.model_selection import train_test_split

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

# from tqdm.notebook import tqdm, trange
# from torch.utils.data import Dataset, DataLoader

# import matplotlib.pyplot as plt

!!! Important notice, while your input needs to be vectors, DO NOT use .csv files, use .npy instead, or else you cannot read a correct vectors.

In [36]:
df_demo = np.load('mtg-jamendo-dataset/extracted_features/features_00.npy', allow_pickle=True)

In [37]:
# Check if the array contains dictionaries and try to access the first one
if df_demo.size > 0 and isinstance(df_demo[0], dict):
    # Access keys from the first dictionary
    df_cols = df_demo[0].keys()
    print("Keys:", df_cols)
else:
    print("The array does not contain dictionaries or is empty.")

Keys: dict_keys(['file_id', 'chroma_stft', 'chroma_cqt', 'chroma_vqt', 'melspectrogram', 'mfcc', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast', 'spectral_flatness', 'spectral_rolloff', 'tonnetz', 'zero_crossing_rate', 'tempo'])


In [42]:
df_00 = pd.DataFrame(list(df_demo))
df_00.head()

,file_id,chroma_stft,chroma_cqt,chroma_vqt,melspectrogram,mfcc,rms,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_flatness,spectral_rolloff,tonnetz,zero_crossing_rate,tempo
0,1009600,"[[0.37275773, 0.68224937, 0.5186986, 0.1486953...","[[0.47720718, 0.36940077, 0.32631412, 0.273587...","[[0.77509445, 0.73389447, 0.5826401, 0.6072246...","[[1.215075e-10, 1.5663591e-08, 2.0346082e-05, ...","[[-518.17584, -518.17584, -430.29666, -233.213...","[[3.7124648e-06, 9.153164e-06, 0.0537972, 0.13...","[[5293.672418761745, 4573.269319269123, 1819.8...","[[3229.8423973088693, 3708.0588948964246, 2262...","[[16.032023805245878, 2.0967393158772794, 9.17...","[[0.27116922, 0.026340809, 0.00017949544, 2.05...","[[9108.544921875, 9625.341796875, 2605.5175781...","[[0.0059788367865704, 0.008654554367897874, 0....","[[0.1416015625, 0.21875, 0.2490234375, 0.19433...",[120.18531976744185]
1,1012000,"[[0.71912336, 0.34964237, 0.654007, 0.46289754...","[[0.088374764, 0.09587196, 0.105847985, 0.1066...","[[0.20937066, 0.17422894, 0.18131544, 0.166340...","[[1.3132918, 8.80866, 16.123262, 16.627686, 57...","[[-352.9353, -247.63692, -213.43446, -219.3783...","[[0.09320916, 0.10855302, 0.14824298, 0.200096...","[[1055.1259598395823, 1136.9553366293653, 1072...","[[1950.6302314848617, 2021.920755201838, 1880....","[[17.6881755578814, 14.569273871186837, 24.778...","[[1.3447055e-05, 4.011983e-06, 2.1252313e-06, ...","[[1875.0, 2320.3125, 2203.125, 1921.875, 1476....","[[0.12766291574002142, 0.1272126794345104, 0.1...","[[0.00830078125, 0.01318359375, 0.0146484375, ...",[108.17307692307692]
2,1014400,"[[0.5011056, 0.27523834, 0.21895956, 0.2125652...","[[0.9597693, 0.8890338, 0.8020399, 0.77287316,...","[[0.66553617, 0.7909594, 0.8403489, 0.7374317,...","[[1.7201235e-07, 3.4058945e-07, 2.9828874e-07,...","[[-560.13873, -558.5219, -558.48444, -558.5812...","[[7.0476e-05, 8.8548826e-05, 0.00010169729, 0....","[[4090.0302295845527, 4021.4778127252343, 4007...","[[3633.6274440913207, 3641.0627559193517, 3702...","[[7.905521530213726, 16.03551469550498, 8.3952...","[[0.0057692244, 0.002627876, 0.00211613, 0.001...","[[8914.74609375, 8871.6796875, 9022.412109375,...","[[0.09851414676052478, 0.1180710628465129, 0.1...","[[0.0625, 0.07666015625, 0.0927734375, 0.05712...",[114.84375]
3,1028900,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.32319668, 0.40636298, 0.3495337, 0.3717830...","[[1.0, 1.0, 1.0, 0.9964585, 0.85148233, 0.7302...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-624.7657, -624.7657, -624.7657, -624.7657, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[9.648455469808276, 9.648455469808276, 9.6484...","[[1.0000007, 1.0000007, 1.0000007, 1.0000007, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.006949342827602739, 0.0915306827762488, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[117.45383522727273]
4,1032700,"[[1.0, 1.0, 0.9499455, 0.6053521, 0.6089698, 0...","[[0.7084062, 0.8268159, 0.69419724, 0.67049265...","[[0.62421703, 0.83904797, 0.6819323, 0.6426657...","[[14.216845, 97.02242, 165.52014, 143.62022, 6...","[[-54.46705, 2.1966453, -8.602338, -22.928173,...","[[0.24664032, 0.30014926, 0.3286325, 0.3264826...","[[3675.2266699547836, 3339.7049949659645, 3204...","[[3367.277827858562, 3357.732217469801, 3724.9...","[[10.893392468491456, 16.388521134894027, 18.6...","[[0.003496554, 0.0019605067, 0.00086086366, 0....","[[6481.494140625, 6201.5625, 6826.025390625, 8...","[[-0.015362937944589974, -0.005755746579547903...","[[0.080078125, 0.09521484375, 0.10498046875, 0...",[109.95678191489361]


In [ ]:
# Process each folder and file, starting from '05'
start_folder = '05'  # Define the folder from which to start processing
start_processing = False

for folder in sorted(os.listdir(dataset_path)):
    if folder >= start_folder:
        start_processing = True
    if not start_processing:
        continue

    folder_path = os.path.join(dataset_path, folder)
    all_features = []  # List to hold all features for this folder

    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder}")
        for file in os.listdir(folder_path):
            if file.endswith('.mp3'):
                file_path = os.path.join(folder_path, file)
                file_id = file.split('.')[0].split('_')[0]
                try:
                    features = extract_features(file_path, file_id)
                    all_features.append(features)
                    print(f"Processed file: {file}")
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

        # Convert feature list to numpy array and save as .npy file
        feature_array = np.array(all_features)
        np.save(os.path.join(output_path, f'features_{folder}.npy'), feature_array)
        print(f"Features saved to '{output_path}/features_{folder}.npy'")

print("Feature extraction complete for all folders.")